<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_25_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Посчитать количество установок по месяцам.

In [ ]:
query = """select
	DATE_TRUNC('month', first_seen) as install_month,
	COUNT(distinct device_code) as installs_number
from app_sessions
group by DATE_TRUNC('month', first_seen)
order by installs_number;
"""

2. Найти пользователей, установивших приложение в марте, и сделавших покупку в течение 30 дней.

In [ ]:
query = """
select ol.user_uuid
from orders_log ol
join devices_users_map dum on ol.user_uuid = dum.user_uuid
join (
	select device_code, first_seen
	from app_sessions
	where date_trunc('month', first_seen) = DATE '2024-03-01'
) s_i on dum.device_code   = s_i.device_code
where
	ol.order_time::DATE <= s_i.first_seen + interval '30 days'
"""

3. Вычислить разницу между первой и последней покупкой по каждому пользователю.

In [ ]:
q1 = """
SELECT
    user_uuid,
    MIN(order_time) AS first_order,
    MAX(order_time) AS last_order,
    MAX(order_time) - MIN(order_time) AS diff_days
FROM orders_log
GROUP BY user_uuid;
"""

q2 = """
SELECT DISTINCT user_uuid,
       MIN(order_time) OVER (PARTITION BY user_uuid)::DATE AS first_purchase,
       MAX(order_time) OVER (PARTITION BY user_uuid)::DATE AS last_purchase,
       (MAX(order_time) OVER (PARTITION BY user_uuid)::DATE -
        MIN(order_time) OVER (PARTITION BY user_uuid)::DATE) AS days_between
FROM orders_log;
"""

4. Построить когортную таблицу: месяц установки × количество покупателей через 0, 1, 2 месяца.


In [ ]:
query = """
select
	cohort_month,
	EXTRACT(MONTH FROM AGE(purchase_month, cohort_month))::INT AS month_offset,
	COUNT(distinct user_uuid) as buyers
from (
	select
		date_trunc('month', as2.first_seen) as cohort_month,
		date_trunc('month', ol.order_time) as purchase_month,
		ol.user_uuid
	from orders_log ol
	join devices_users_map dum on dum.user_uuid = ol.user_uuid
	join app_sessions as2 on as2.device_code = dum.device_code
) s_i
where purchase_month >= cohort_month and
EXTRACT(MONTH FROM AGE(purchase_month, cohort_month))::INT in (0, 1, 2)
group by cohort_month, month_offset
order by cohort_month, month_offset;
"""